In [1]:
from dotenv import load_dotenv
import os
from urllib.parse import quote_plus
from dotenv import load_dotenv
from sqlalchemy import create_engine
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import AzureChatOpenAI

In [2]:
load_dotenv()

True

In [3]:
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_api_version = os.environ["OPENAI_API_VERSION"]

database_connection_string = os.environ["SQL_DATABASE_CONNECTION_STRING"]
database_password = os.environ["SQL_DATABASE_PASSWORD"]

In [11]:
def database_connector(database_connection_string, database_password, schema): 
    try: 
        db_engine = create_engine(database_connection_string % quote_plus(database_password))
        db = SQLDatabase(db_engine, view_support=True, schema=schema)
        print(db.dialect)
        print(db.get_usable_table_names())
        db.run("select convert(varchar(25), getdate(), 120)")
        
        return db
    except Exception as e: 
        print(f"Connection to database failed. {e}")


def get_llm_client(model_name = "gpt-4", deployment = "gpt-4"): 

    azurellm = AzureChatOpenAI(
        model_name=model_name,
        deployment_name=deployment 
    )

    return azurellm

def get_sql_agent(azurellm, database):
    toolkit = SQLDatabaseToolkit(db=database, llm=azurellm)

    agent_executor = create_sql_agent(
        llm=azurellm,
        toolkit=toolkit,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True
    )

    return agent_executor

In [ ]:
db = database_connector(database_connection_string=database_connection_string, database_password=database_password, schema="dbo")
azurellm = get_llm_client()
llm_agent = get_sql_agent(azurellm, db)

In [13]:
base_prompt = """You are a helpful assistant working for a energy supplier. They are keen to understand their demand forcasts to efficiently plan their operations. 
Based on the data provided, you will respond to any questions that an user might have.
For every user question which will be provided below, please return an analysis and a summary of the information you used in your analysis. 
Please dont use pleasantries of any form. Do not hallucinate, if the response is not within the data or you cannot figure it, 
return information not found."""
user_prompt = "based on the weather data provided in the curated table, can you tell me around what times during winter months a energy supplier can expect spike in demands?"
llm_agent.invoke(base_prompt + user_prompt)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: curated_weather_data, raw_weather_dataThe 'curated_weather_data' table seems more relevant for getting cleaned and useful weather data. I need to check the schema of this table to see what columns are available, particularly those that could relate to time and the winter months.
Action: sql_db_schema
Action Input: curated_weather_data
CREATE TABLE dbo.curated_weather_data (
	record_id NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	temperature FLOAT(53) NULL, 
	relative_humidity FLOAT(53) NULL, 
	precipitation FLOAT(53) NULL, 
	wind_direction FLOAT(53) NULL, 
	wind_speed FLOAT(53) NULL, 
	wind_gust FLOAT(53) NULL, 
	pressure FLOAT(53) NULL, 
	sun_shine FLOAT(53) NULL, 
	weather_condition_code INTEGER NULL, 
	snow FLOAT(53) NULL, 
	dew_point NVARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	time DATETIME2 NULL
)

/*
3 rows from curated_weather_data table:
record_id	temperature	relat

{'input': 'You are a helpful assistant working for a energy supplier. They are keen to understand their demand forcasts to efficiently plan their operations. \nBased on the data provided, you will respond to any questions that an user might have.\nFor every user question which will be provided below, please return an analysis and a summary of the information you used in your analysis. \nPlease dont use pleasantries of any form. Do not hallucinate, if the response is not within the data or you cannot figure it, \nreturn information not found.based on the weather data provided in the curated table, can you tell me around what times during winter months a energy supplier can expect spike in demands?',
 'output': 'During winter months, the energy supplier can expect a spike in demands early in the morning, around 6:00 AM to 8:00 AM, when the temperatures are at their lowest based on the available data.'}